In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

from category_encoders.cat_boost import CatBoostEncoder

Using TensorFlow backend.


In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop('id', axis=1)

In [4]:
# create training data by stratified Kfold to add noises to the training set
en_cols = X_train.columns.tolist()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
X_train_l = []
X_test_l = []
y_train_l = []
for train_index, test_index in kf.split(X_train, y_train):
    X_dev = X_train.iloc[train_index]
    y_dev = y_train.iloc[train_index]
    X_val = X_train.iloc[test_index]
    y_val = y_train.iloc[test_index]
    cbe_temp = CatBoostEncoder(cols=en_cols).fit(X_dev, y_dev)
    X_train_l.append(cbe_temp.transform(X_val))
    X_test_l.append(cbe_temp.transform(X_test))
    y_train_l.append(y_val)

In [5]:
# finalise the training data
X_train = pd.concat(X_train_l)
y_train = pd.concat(y_train_l)
X_test = sum(X_test_l) / 10

In [9]:
# logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

c:\users\p1319163\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# xgboost 
xgb = XGBClassifier(max_depth=10, 
                    n_estimators=1000, 
                    learning_rate=0.01, 
                    n_jobs=7, 
                    random_state=42, 
                    scale_pos_weight=2.268)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=None, n_estimators=1000,
              n_jobs=7, nthread=None, objective='binary:logistic',
              random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=2.268, seed=None, silent=True, subsample=1)

In [11]:
# Adaboost
ada = AdaBoostClassifier(n_estimators=5000, learning_rate=0.01, random_state=42)
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.01,
                   n_estimators=5000, random_state=42)

In [15]:
# neural network
es = EarlyStopping(monitor='val_auroc', mode='max', verbose=1, min_delta=0.001, patience=100)
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


nn = Sequential()
nn.add(Dense(23, input_dim=23, activation='relu'))
nn.add(Dense(12, activation='relu'))
nn.add(Dense(6, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auroc])
nn.fit(X_train, y_train, 
       batch_size=256, 
       epochs=1000, 
       verbose=1, 
       callbacks=[es], 
       validation_split=0.2)

Train on 240000 samples, validate on 60000 samples
Epoch 1/1000
240000/240000 [==============================] - 3s 11us/step - loss: 0.5875 - auroc: 0.6064 - val_loss: 0.5434 - val_auroc: 0.7430
Epoch 2/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.5227 - auroc: 0.7611 - val_loss: 0.5102 - val_auroc: 0.7728
Epoch 3/1000
240000/240000 [==============================] - 3s 10us/step - loss: 0.5038 - auroc: 0.7832 - val_loss: 0.4980 - val_auroc: 0.7867
Epoch 4/1000
240000/240000 [==============================] - 3s 10us/step - loss: 0.4957 - auroc: 0.7925 - val_loss: 0.4929 - val_auroc: 0.7921
Epoch 5/1000
240000/240000 [==============================] - 3s 11us/step - loss: 0.4911 - auroc: 0.7961 - val_loss: 0.4905 - val_auroc: 0.7944
Epoch 6/1000
240000/240000 [==============================] - 3s 11us/step - loss: 0.4897 - auroc: 0.7980 - val_loss: 0.4898 - val_auroc: 0.7955
Epoch 7/1000
240000/240000 [==============================] - 3s 10us/step - lo

Epoch 57/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4860 - auroc: 0.8010 - val_loss: 0.4899 - val_auroc: 0.7982
Epoch 58/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4866 - auroc: 0.8010 - val_loss: 0.4889 - val_auroc: 0.7980
Epoch 59/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4862 - auroc: 0.8010 - val_loss: 0.4870 - val_auroc: 0.7981
Epoch 60/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4862 - auroc: 0.8011 - val_loss: 0.4877 - val_auroc: 0.7981
Epoch 61/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4862 - auroc: 0.8012 - val_loss: 0.4891 - val_auroc: 0.7980
Epoch 62/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4863 - auroc: 0.8011 - val_loss: 0.4879 - val_auroc: 0.7981
Epoch 63/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4855 - auroc: 0.8010 - val_loss: 0.487

240000/240000 [==============================] - 3s 10us/step - loss: 0.4853 - auroc: 0.8015 - val_loss: 0.4876 - val_auroc: 0.7986
Epoch 114/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4859 - auroc: 0.8012 - val_loss: 0.4865 - val_auroc: 0.7985
Epoch 115/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4856 - auroc: 0.8014 - val_loss: 0.4873 - val_auroc: 0.7984
Epoch 116/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4855 - auroc: 0.8014 - val_loss: 0.4865 - val_auroc: 0.7985
Epoch 117/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4855 - auroc: 0.8016 - val_loss: 0.4888 - val_auroc: 0.7983
Epoch 118/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4856 - auroc: 0.8014 - val_loss: 0.4869 - val_auroc: 0.7983
Epoch 119/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4853 - auroc: 0.8014 - val_loss: 0.4866 - val_

In [17]:
nn.fit(X_train, y_train, 
       batch_size=256, 
       epochs=1000, 
       verbose=1, 
       callbacks=[es], 
       validation_split=0.2)

Train on 240000 samples, validate on 60000 samples
Epoch 1/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4848 - auroc: 0.8019 - val_loss: 0.4861 - val_auroc: 0.7989
Epoch 2/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4849 - auroc: 0.8019 - val_loss: 0.4905 - val_auroc: 0.7989
Epoch 3/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4846 - auroc: 0.8019 - val_loss: 0.4862 - val_auroc: 0.7987
Epoch 4/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4848 - auroc: 0.8020 - val_loss: 0.4868 - val_auroc: 0.7988
Epoch 5/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4847 - auroc: 0.8020 - val_loss: 0.4863 - val_auroc: 0.7989
Epoch 6/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4844 - auroc: 0.8019 - val_loss: 0.4866 - val_auroc: 0.7987
Epoch 7/1000
240000/240000 [==============================] - 2s 9us/step - loss: 0.4

Epoch 57/1000
240000/240000 [==============================] - 3s 11us/step - loss: 0.4843 - auroc: 0.8020 - val_loss: 0.4870 - val_auroc: 0.7989
Epoch 58/1000
240000/240000 [==============================] - 3s 11us/step - loss: 0.4846 - auroc: 0.8019 - val_loss: 0.4889 - val_auroc: 0.7989
Epoch 59/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4844 - auroc: 0.8019 - val_loss: 0.4876 - val_auroc: 0.7990
Epoch 60/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4844 - auroc: 0.8019 - val_loss: 0.4863 - val_auroc: 0.7988
Epoch 61/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4845 - auroc: 0.8020 - val_loss: 0.4864 - val_auroc: 0.7989
Epoch 62/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4847 - auroc: 0.8021 - val_loss: 0.4862 - val_auroc: 0.7988
Epoch 63/1000
240000/240000 [==============================] - 2s 10us/step - loss: 0.4846 - auroc: 0.8019 - val_loss: 0.486

In [19]:
# get prediction results for all models
lr_out = pd.DataFrame({'id': test['id'], 'target': lr.predict_proba(X_test)[:,1]})
xgb_out = pd.DataFrame({'id': test['id'], 'target': xgb.predict_proba(X_test)[:,1]})
ada_out = pd.DataFrame({'id': test['id'], 'target': ada.predict_proba(X_test)[:,1]})
nn_out = pd.DataFrame({'id': test['id'], 'target':[i[0] for i in nn.predict_proba(X_test)]})

In [20]:
# ensemble results from these 4 models -> 0.80481
ens_out = (lr_out + xgb_out + nn_out) / 3 
ens_out['id'] = ens_out['id'].astype(int)

In [21]:
ens_out.to_csv("../submissions/enm-cat-all-noise-submission.csv", index=False)